<a href="https://colab.research.google.com/github/MHoseinHoushmand/Clustering_by_SLFA/blob/main/Clustering_by_SLFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import pdb
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

categories = [
    "alt.atheism",
    "comp.graphics",
    "sci.space",
   # "sci.med",
    "rec.sport.hockey",
]

dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)
answers_list = []
labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"{len(dataset.data)} documents - {true_k} categories")

3758 documents - 4 categories


In [ ]:
from sklearn.datasets import fetch_20newsgroups

# بارگیری دیتاست 20newsgroups
data_with_names = fetch_20newsgroups()

# نمایش نام دسته‌ها برای هر دیتا
for i in range(len(data_with_names.target)):
    label = data_with_names.target[i]
    print(label)
    category_name = data_with_names.target_names[label]
    print(f"نام دسته‌ی متناظر با دیتا {i+1}: {category_name}")

Streaming output truncated to the last 5000 lines.
7
نام دسته‌ی متناظر با دیتا 8815: rec.autos
6
نام دسته‌ی متناظر با دیتا 8816: misc.forsale
7
نام دسته‌ی متناظر با دیتا 8817: rec.autos
18
نام دسته‌ی متناظر با دیتا 8818: talk.politics.misc
16
نام دسته‌ی متناظر با دیتا 8819: talk.politics.guns
8
نام دسته‌ی متناظر با دیتا 8820: rec.motorcycles
4
نام دسته‌ی متناظر با دیتا 8821: comp.sys.mac.hardware
8
نام دسته‌ی متناظر با دیتا 8822: rec.motorcycles
8
نام دسته‌ی متناظر با دیتا 8823: rec.motorcycles
17
نام دسته‌ی متناظر با دیتا 8824: talk.politics.mideast
11
نام دسته‌ی متناظر با دیتا 8825: sci.crypt
15
نام دسته‌ی متناظر با دیتا 8826: soc.religion.christian
13
نام دسته‌ی متناظر با دیتا 8827: sci.med
3
نام دسته‌ی متناظر با دیتا 8828: comp.sys.ibm.pc.hardware
4
نام دسته‌ی متناظر با دیتا 8829: comp.sys.mac.hardware
10
نام دسته‌ی متناظر با دیتا 8830: rec.sport.hockey
12
نام دسته‌ی متناظر با دیتا 8831: sci.electronics
10
نام دسته‌ی متناظر با دیتا 8832: rec.sport.hockey
5
نام دسته‌ی متناظر با دیتا

In [ ]:
print (dataset.data[800],dataset.target_names[dataset.target[800]],dataset.target[800])


There is really nothing inherently wrong with it but they tried it just
a little over a decade ago, and noone showed up for the early rounds in
the playoffs...whereas soon after they went to the divisional set-up
arenas were mostly filled in the early rounds.  

The empirical evidence of the last two decades is that more people will
show up to see the Bruins play the Sabre umpteen times than see the
Bruins play the Captials in the first round.

Maybe hockey has increased in popularity sufficiently that this will
no longer be the case.  The experiment is worthwhile with the uneven
distribution of the expansion teams, but I prefer the divisional 
playoff. rec.sport.hockey 2


In [ ]:

def docs_as_tfidf(docs):
  vectorizer = TfidfVectorizer(
     max_df=0.5,
     min_df=5,
     stop_words="english",
  )

  docs_vector = vectorizer.fit_transform(docs)
  return docs_vector.toarray()

In [ ]:
population_size = 120 # Frogs number
memplex_num = 12 #define as m
memplex_size = 10 #define as n
max_iteration = 150
memplex_iteration = 8
docs = dataset.data
docs_vector = docs_as_tfidf(docs)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def cosin_sim(a,b):
   return cosine_similarity([a], [b])[0][0]

In [ ]:
from numpy.linalg import norm
def SSE(cluster,doc_mean):
  size = len(cluster)
  sse=0
  for doc in cluster:
    sse += cosin_sim(doc,doc_mean)**2
  sse = sse/size
  return sse

In [ ]:
from numpy.linalg import norm
def BC(doc_means):
   BC=0
   size = len(doc_means)
   for i in range(size):
      for j in range(i+1,size):
          BC += cosin_sim(doc_means[i],doc_means[j])**2
   return BC

In [ ]:
def WC(clusters):
    WC = 0
    for cluster in clusters:
        doc_mean = np.average(cluster, axis=0)
        WC += SSE(cluster,doc_mean)
    return WC

In [ ]:
def build_clusters(answer,docs_vector,clusters_size):
   clusters = []
   for i in range(clusters_size):
       clusters.append([])
   for j in range(len(answer)):
       if -1 < answer[j]:
        clusters[answer[j]].append(docs_vector[j])
   return clusters

In [ ]:
def fitness(answer,docs_vector,clusters_size):
   doc_means = []
 #  pdb.runcall(build_clusters,answer,docs_vector,clusters_size)
   clusters = build_clusters(answer,docs_vector,clusters_size)
   for i in range(clusters_size):
       doc_means.append(
          np.average(clusters[i], axis=0)
       )
   wc = WC(clusters)
   bc = BC(doc_means)
   fitness = wc/bc
   return fitness

In [ ]:
import random
def cross_over(answer_a,answer_b):
    size = len(answer_a)
    output = []
    for i in range(size):
       choice = random.choice([0,1])
       if choice == 0:
          output.append(answer_a[i])
       else:
          output.append(answer_b[i])
    return tuple(output)

In [ ]:
def best_and_worst(answers):
     best =  max(answers, key=answers.get)
     worst = min(answers, key=answers.get)
     return tuple(best) , tuple(worst)

In [ ]:
def global_best(memplexes):
     local_bests = {}
     for memplex in memplexes:
         local_best =  max(memplex, key=memplex.get)
         local_bests[local_best]= memplex[local_best]
     global_best = max(local_bests, key=local_bests.get)
     return global_best, local_bests[global_best]

In [ ]:
def keys_to_remove(keys , dict):
   for k in keys:
      if k in dict:
          dict.pop(k)
   return dict

In [ ]:
def mutation(global_best,clusters_size):
    new_ans = list(global_best)
    size = int(len(global_best)/4)
    indexes = np.random.choice(np.arange(0,len(global_best)), size=size, replace=False)
    values= [random.randint(0, 3) for _ in range(size)]
    for i in range(size):
      new_ans[indexes[i]] = values[i]
    return tuple(new_ans)

In [ ]:
def Create_memplexes(population,memplex_num):
     memplexes = []
     keys = list(population.keys())
     population_size = len(population)
     for i in range(memplex_num):
         memplexes.append({})
     for i in range(population_size):
         memplexes[i % memplex_num][keys[i]] = population[keys[i]]
     return memplexes

In [ ]:
def shufeling(memplexes):
    output = {}
    for memplex in memplexes:
        output.update(memplex)
    return output

In [ ]:
def Trues(g_best):
  true = 0
  size = len(dataset.data)
  for i in range(size):
     if labels[i] == g_best[i]:
        true = true +1
  print(true)

In [ ]:
def Check_num(memplexes,position):
   for memplex in memplexes:
      if len(memplex)< 5:
        print("errrrrrrrrrrrorrrrrrrrrrrrr!"+ position)
        pdb.set_trace()

In [ ]:
print(list(labels))

[0, 1, 1, 3, 1, 1, 2, 2, 2, 0, 1, 2, 3, 2, 0, 3, 3, 1, 1, 2, 3, 2, 1, 1, 2, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2, 2, 2, 0, 3, 0, 1, 2, 0, 1, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0, 2, 3, 1, 0, 2, 1, 2, 2, 1, 3, 3, 3, 0, 1, 3, 1, 0, 0, 0, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 1, 1, 1, 0, 3, 3, 2, 1, 0, 3, 3, 2, 3, 1, 1, 2, 0, 2, 2, 1, 3, 1, 1, 0, 0, 0, 2, 3, 2, 2, 3, 3, 0, 0, 0, 1, 2, 0, 1, 3, 0, 1, 0, 1, 0, 3, 3, 3, 3, 3, 0, 1, 0, 2, 0, 0, 0, 1, 2, 3, 3, 2, 2, 1, 1, 0, 0, 0, 2, 1, 2, 1, 0, 1, 1, 2, 3, 1, 2, 3, 0, 0, 3, 2, 1, 0, 3, 1, 2, 0, 2, 2, 3, 2, 3, 2, 1, 2, 3, 3, 2, 2, 0, 1, 2, 2, 2, 2, 1, 1, 3, 2, 2, 3, 3, 2, 1, 0, 2, 1, 0, 2, 3, 2, 2, 3, 0, 2, 1, 1, 0, 2, 2, 1, 2, 1, 3, 2, 3, 1, 2, 2, 1, 0, 2, 1, 2, 3, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 2, 3, 2, 1, 2, 1, 1, 0, 1, 3, 0, 1, 1, 1, 3, 0, 2, 3, 0, 2, 1, 2, 0, 0, 2, 3, 2, 2, 1, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 1, 0, 2, 3, 2, 3, 1, 0, 0, 1, 1, 1, 1, 0, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 3, 2, 2, 3, 3, 2, 3, 3, 0, 1, 3, 1, 1, 2, 2, 2, 3, 1, 1, 2, 0, 0, 0, 

In [24]:
import operator
def frog_leaping_search(docs_vector,cluster_size):
     answers=np.random.randint(0, cluster_size, size=(population_size , len(dataset.data)))
     print(list(answers[0]))
     population = {}
     i=0
     for answer in answers:
       # pdb.runcall(fitness,answer,docs_vector,cluster_size)
        i+=1
        population[tuple(answer)] = fitness(answer,docs_vector,cluster_size)
        print(i,population[tuple(answer)])

     for i in range(max_iteration):
        population = dict( sorted(population.items(), key=operator.itemgetter(1), reverse=True))
   #    pdb.runcall(Create_memplexes,population, memplex_num)
        memplexes = Create_memplexes(population, memplex_num)
        population.clear()
      #  pdb.set_trace()
        for j in range(memplex_num):
            print(i,j,len(memplexes[j]))
            sub_memplex = dict(random.sample(list(memplexes[j].items()),k=5))
            memplexes[j] =  keys_to_remove(sub_memplex.keys(),memplexes[j])
            for k in range(memplex_iteration):
                #pdb.runcall(best_and_worst,sub_memplex)

           #     for m in sub_memplex:
            #        print(list(m))                ans_best, ans_worst = best_and_worst(sub_memplex)
                ans_out = cross_over(ans_best,ans_worst)
                fitness_out = fitness(ans_out,docs_vector,cluster_size)
                #sec B
                ###############################################
                if len(sub_memplex)< 5:
                              print("errrrrrrrrrrrorrrrrrrrrrrrrB")
                              pdb.set_trace()
              ###############################################
              #pdb.runcall(best_and_worst,sub_memplex)
                print("############")


                if (sub_memplex[ans_worst]<fitness_out):
                    del sub_memplex[ans_worst]
                    sub_memplex[ans_out] = fitness_out
                    #sec C
                   ###############################################
                    if len(sub_memplex)< 5:
                              print("errrrrrrrrrrrorrrrrrrrrrrrrC")
                              pdb.set_trace()
                  ###############################################
                else:
                   # pdb.runcall(global_best,memplexes)
                    print("################")
                    for m in sub_memplex:
                         print(list(m))
                    g_best, g_value = global_best(memplexes)
                    ans_out = cross_over(g_best,ans_worst)
                    #sec D
                    ###############################################
                    if len(sub_memplex)< 5:
                          print("errrrrrrrrrrrorrrrrrrrrrrrrD")
                          pdb.set_trace()
                    ###############################################
                    fitness_out = fitness(ans_out,docs_vector,cluster_size)
                    print("#############")
                    for m in sub_memplex:
                         print(list(m))
                    if (sub_memplex[ans_worst] < fitness_out):
                        del sub_memplex[ans_worst]
                        sub_memplex[ans_out] = fitness_out
                        #sec E
                        ###############################################
                        if len(sub_memplex)< 5:
                              print("errrrrrrrrrrrorrrrrrrrrrrrrE")
                              pdb.set_trace()
                    ###############################################
                    else:
                   #     print("#########")
                    #    for m in sub_memplex:
                     #        print(list(m))
                        del sub_memplex[ans_worst]
                     #   pdb.runcall(mutation,g_best,cluster_size)
                        ans_out = mutation(g_best,cluster_size)
                        fitness_out = fitness(ans_out,docs_vector,cluster_size)
                        sub_memplex[ans_out] = fitness_out
                        #sec F
                        ###############################################
                        if len(sub_memplex)< 5:
                              print("errrrrrrrrrrrorrrrrrrrrrrrrF")
                              pdb.set_trace()
                       ###############################################
        #     pdb.runcall(join_dicts,memplexes[j],sub_memplex)
            memplexes[j].update(sub_memplex)
        g_best, g_value = global_best(memplexes)
       # pdb.runcall(show_result,g_best)
        answers_list.append(g_best)
        print(g_best)
        print(g_value)
        population = shufeling(memplexes)
     return g_best, g_value, population

In [22]:
g_best, g_value,population = frog_leaping_search(docs_vector,4)
true = 0
size = len(dataset.data)
print(labels)
print(list(g_best))

[1, 3, 1, 3, 0, 2, 1, 1, 3, 3, 0, 3, 0, 1, 2, 1, 3, 3, 3, 1, 1, 1, 3, 0, 1, 1, 0, 0, 3, 3, 1, 3, 0, 3, 3, 2, 2, 2, 1, 3, 3, 1, 0, 2, 1, 3, 3, 2, 1, 1, 2, 0, 2, 3, 2, 2, 1, 0, 0, 0, 3, 2, 2, 1, 2, 0, 3, 2, 1, 1, 1, 0, 2, 1, 1, 1, 3, 0, 0, 3, 0, 1, 2, 0, 2, 0, 2, 1, 0, 2, 2, 2, 2, 3, 2, 2, 2, 3, 1, 0, 1, 0, 1, 2, 2, 1, 0, 0, 0, 1, 0, 2, 0, 2, 1, 1, 3, 3, 3, 2, 0, 1, 0, 0, 1, 3, 2, 3, 3, 0, 0, 0, 3, 1, 2, 1, 1, 0, 2, 1, 1, 0, 2, 3, 2, 0, 0, 3, 1, 1, 3, 0, 3, 1, 0, 2, 3, 2, 1, 3, 0, 1, 3, 0, 3, 0, 2, 3, 3, 2, 2, 1, 1, 0, 0, 2, 0, 1, 3, 2, 0, 3, 0, 0, 1, 3, 1, 0, 1, 2, 3, 1, 0, 1, 0, 1, 3, 1, 0, 2, 0, 2, 0, 1, 1, 1, 3, 0, 2, 2, 1, 1, 0, 2, 0, 3, 0, 0, 1, 3, 3, 0, 0, 2, 1, 0, 0, 3, 3, 0, 0, 1, 1, 1, 0, 3, 0, 3, 2, 2, 2, 1, 1, 2, 3, 2, 3, 0, 3, 3, 2, 3, 1, 0, 1, 2, 3, 0, 0, 0, 3, 0, 0, 2, 3, 3, 3, 0, 3, 0, 1, 1, 2, 1, 0, 0, 1, 3, 0, 2, 3, 0, 0, 3, 2, 1, 1, 0, 3, 0, 0, 2, 0, 1, 2, 0, 0, 1, 3, 0, 3, 2, 1, 2, 0, 3, 1, 0, 3, 0, 1, 2, 0, 0, 1, 2, 2, 3, 0, 2, 0, 1, 2, 3, 1, 0, 3, 2, 2, 2, 1, 2, 0, 

KeyboardInterrupt: ignored

In [ ]:
#print(list(labels))
print(list(g_best))

In [ ]:
print(g_value)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt



def show_result(g_best):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.5, max_features=1000, stop_words='english')
    lsa = TruncatedSVD(n_components=2)
    lsa_matrix = lsa.fit_transform(docs_vector)
    plt.figure(figsize=(10, 8))
    plt.scatter(lsa_matrix[:, 0], lsa_matrix[:, 1], c=list(g_best), cmap='viridis')

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt



tfidf_vectorizer = TfidfVectorizer(max_df=0.5, max_features=1000, stop_words='english')

lsa = TruncatedSVD(n_components=2)
lsa_matrix = lsa.fit_transform(docs_vector)

plt.figure(figsize=(10, 8))
plt.scatter(lsa_matrix[:, 0], lsa_matrix[:, 1], c=list(labels), cmap='viridis')

In [ ]:
show_result(answers_list[1])

In [ ]:
show_result(answers_list[2])

In [ ]:
show_result(answers_list[3])

In [ ]:
show_result(answers_list[4])

In [ ]:
show_result(answers_list[5])

In [ ]:
show_result(answers_list[6])

In [ ]:
show_result(answers_list[7])

In [ ]:
show_result(answers_list[8])

In [ ]:
show_result(answers_list[9])

In [ ]:
show_result(answers_list[10])

In [ ]:
show_result(answers_list[11])

In [ ]:
show_result(answers_list[12])

In [ ]:
show_result(answers_list[13])

In [ ]:
show_result(answers_list[14])

In [ ]:
show_result(answers_list[15])

In [ ]:
show_result(answers_list[16])

In [ ]:
show_result(answers_list[17])

In [ ]:
show_result(answers_list[18])

In [ ]:
show_result(answers_list[19])

In [ ]:
show_result(answers_list[20])

In [ ]:
show_result(answers_list[21])

In [ ]:
show_result(answers_list[22])

In [ ]:
show_result(answers_list[23])

In [ ]:
show_result(answers_list[24])

In [ ]:
show_result(answers_list[25])

In [ ]:
show_result(answers_list[26])

In [ ]:
show_result(answers_list[27])

In [ ]:
show_result(answers_list[28])

In [ ]:
show_result(answers_list[29])

In [ ]:
show_result(answers_list[30])

In [ ]:
show_result(answers_list[31])

In [ ]:
show_result(answers_list[32])

In [ ]:
show_result(answers_list[33])

In [ ]:
show_result(answers_list[34])

In [ ]:
show_result(answers_list[35])

In [ ]:
show_result(answers_list[36])

In [ ]:
show_result(answers_list[37])

In [ ]:
show_result(answers_list[38])

In [ ]:
show_result(answers_list[39])

In [ ]:
show_result(answers_list[40])

In [ ]:
show_result(answers_list[41])

In [ ]:
show_result(answers_list[42])

In [ ]:
show_result(answers_list[43])

In [ ]:
show_result(answers_list[44])

In [ ]:
show_result(answers_list[45])

In [ ]:
show_result(answers_list[46])

In [ ]:
show_result(answers_list[47])

In [ ]:
show_result(answers_list[48])

In [ ]:
show_result(answers_list[49])

In [ ]:
show_result(answers_list[0])

In [ ]:
def more_iteration(population,cluster_size):
    for i in range(max_iteration):
        population = dict( sorted(population.items(), key=operator.itemgetter(1), reverse=True))
   #    pdb.runcall(Create_memplexes,population, memplex_num)
        memplexes = Create_memplexes(population, memplex_num)
        population.clear()
      #  pdb.set_trace()
        for j in range(memplex_num):
            print(i,j,len(memplexes[j]))
            sub_memplex = dict(random.sample(list(memplexes[j].items()),k=5))
            memplexes[j] =  keys_to_remove(sub_memplex.keys(),memplexes[j])
            for k in range(memplex_iteration):
                #pdb.runcall(best_and_worst,sub_memplex)
                ans_best, ans_worst = best_and_worst(sub_memplex)
                ans_out = cross_over(ans_best,ans_worst)
                fitness_out = fitness(ans_out,docs_vector,cluster_size)

              # pdb.runcall(best_and_worst,sub_memplex)
               #print(population[ans_worst],population[ans_out])
                if (sub_memplex[ans_worst]<fitness_out):
                    del sub_memplex[ans_worst]
                    sub_memplex[ans_out] = fitness_out
                else:
                   # pdb.runcall(global_best,memplexes)
                    g_best, g_value = global_best(memplexes)
                    ans_out = cross_over(g_best,ans_worst)
                    fitness_out = fitness(ans_out,docs_vector,cluster_size)
                    if (sub_memplex[ans_worst] < fitness_out):
                        del sub_memplex[ans_worst]
                        sub_memplex[ans_out] = fitness_out
                    else:
                        del sub_memplex[ans_worst]
                     #   pdb.runcall(mutation,g_best,cluster_size)
                        ans_out = mutation(g_best,cluster_size)
                        fitness_out = fitness(ans_out,docs_vector,cluster_size)
                        sub_memplex[ans_out] = fitness_out
        #     pdb.runcall(join_dicts,memplexes[j],sub_memplex)
            memplexes[j].update(sub_memplex)
        g_best, g_value = global_best(memplexes)
       # pdb.runcall(show_result,g_best)
        answers_list.append(g_best)
        print(g_value)
        population = shufeling(memplexes)
    gb,gv=g_best, g_value